In [1]:
from labdata.schema import *

[2024-05-12 18:32:02,856][INFO]: Connecting admin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306
[2024-05-12 18:32:03,395][INFO]: Connected admin@churchland-data.cmojfwfr0b9t.us-west-2.rds.amazonaws.com:3306


In [45]:
# TODO: add missing procedures for mice
insertions_query = Procedure() & 'procedure_type = "chronic implant"'
chronic_mice = insertions_query.fetch('subject_name')
chronic_sessions_query = EphysRecording() & [dict(subject_name = c) for c in chronic_mice]
n_sorted_prbs = len(SpikeSorting() & 'parameter_set_num = 5')

chronic_probe_ids = np.unique((EphysRecording.ProbeSetting() & chronic_sessions_query).fetch('probe_id')).astype(int)

insertions = ((pd.DataFrame((EphysRecording.ProbeSetting() & chronic_sessions_query).proj('probe_id').fetch())[
               ['subject_name','probe_id']].drop_duplicates()))


print(f'We performed {len(insertions)} chronic insertions ({len(chronic_probe_ids)} probes) into {len(chronic_mice)} mice and recorded {len(chronic_sessions_query)} sessions.')
print(f'We spike sorted  {n_sorted_prbs} probe*sessions.')
# insertions

We performed 33 chronic insertions (22 probes) into 12 mice and recorded 536 sessions.
We spike sorted  915 probe*sessions.
